In [1]:
from __future__ import print_function
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
train_variant = pd.read_csv("C:/Users/lenovo/Desktop/EC601/Project/data/training_variants")
test_variant = pd.read_csv("C:/Users/lenovo/Desktop/EC601/Project/data/test_variants")
train_text = pd.read_csv("C:/Users/lenovo/Desktop/EC601/Project/data/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text = pd.read_csv("C:/Users/lenovo/Desktop/EC601/Project/data/test_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
train = pd.merge(train_variant, train_text, how='left', on='ID')
train_y = train['Class'].values
train_x = train.drop('Class', axis=1)
train_size=len(train_x)
print('Number of training variants: %d' % (train_size))
# number of train data : 3321

test_x = pd.merge(test_variant, test_text, how='left', on='ID')
test_size=len(test_x)
print('Number of test variants: %d' % (test_size))
# number of test data : 5668

test_index = test_x['ID'].values
all_data = np.concatenate((train_x, test_x), axis=0)
all_data = pd.DataFrame(all_data)
all_data.columns = ["ID", "Gene", "Variation", "Text"]


Number of training variants: 3321
Number of test variants: 5668


In [3]:
all_data[:20]

,ID,Gene,Variation,Text
0,0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...
5,5,CBL,V391I,Oncogenic mutations in the monomeric Casitas B...
6,6,CBL,V430M,Oncogenic mutations in the monomeric Casitas B...
7,7,CBL,Deletion,CBL is a negative regulator of activated recep...
8,8,CBL,Y371H,Abstract Juvenile myelomonocytic leukemia (JM...
9,9,CBL,C384R,Abstract Juvenile myelomonocytic leukemia (JM...


In [4]:
from nltk.corpus import stopwords
from gensim.models.doc2vec import LabeledSentence
from gensim import utils

def constructLabeledSentences(data):
    sentences=[]
    for index, row in data.iteritems():
        sentences.append(LabeledSentence(utils.to_unicode(row).split(), ['Text' + '_%s' % str(index)]))
    return sentences

def textClean(text):
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]    
    text = " ".join(text)
    return(text)
    
def cleanup(text):
    text = textClean(text)
    text= text.translate(str.maketrans("","", string.punctuation))
    return text

allText = all_data['Text'].apply(cleanup)
sentences = constructLabeledSentences(allText)
allText[:20]

E:\Anaconda\anaconda\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


0     cyclindependent kinases cdks regulate variety ...
1     abstract background nonsmall cell lung cancer ...
2     abstract background nonsmall cell lung cancer ...
3     recent evidence demonstrated acquired uniparen...
4     oncogenic mutations monomeric casitas blineage...
5     oncogenic mutations monomeric casitas blineage...
6     oncogenic mutations monomeric casitas blineage...
7     cbl negative regulator activated receptor tyro...
8     abstract juvenile myelomonocytic leukemia jmml...
9     abstract juvenile myelomonocytic leukemia jmml...
10    oncogenic mutations monomeric casitas blineage...
11    noonan syndrome autosomal dominant congenital ...
12    noonan syndrome autosomal dominant congenital ...
13    noonan syndrome autosomal dominant congenital ...
14    oncogenic mutations monomeric casitas blineage...
15    noonan syndrome autosomal dominant congenital ...
16    determine residual cylindrical refractive erro...
17    acquired uniparental disomy aupd common fe

In [5]:
sentences[0]

LabeledSentence(words=['cyclindependent', 'kinases', 'cdks', 'regulate', 'variety', 'fundamental', 'cellular', 'processes', 'cdk10', 'stands', 'one', 'last', 'orphan', 'cdks', 'activating', 'cyclin', 'identified', 'kinase', 'activity', 'revealed', 'previous', 'work', 'shown', 'cdk10', 'silencing', 'increases', 'ets2', 'vets', 'erythroblastosis', 'virus', 'e26', 'oncogene', 'homolog', '2', 'driven', 'activation', 'mapk', 'pathway', 'confers', 'tamoxifen', 'resistance', 'breast', 'cancer', 'cells', 'precise', 'mechanisms', 'cdk10', 'modulates', 'ets2', 'activity', 'generally', 'functions', 'cdk10', 'remain', 'elusive', 'demonstrate', 'cdk10', 'cyclindependent', 'kinase', 'identifying', 'cyclin', 'activating', 'cyclin', 'cyclin', 'm', 'orphan', 'cyclin', 'product', 'fam58a', 'whose', 'mutations', 'cause', 'star', 'syndrome', 'human', 'developmental', 'anomaly', 'whose', 'features', 'include', 'toe', 'syndactyly', 'telecanthus', 'anogenital', 'renal', 'malformations', 'show', 'star', 'synd

In [6]:
from gensim.models import Doc2Vec

Text_INPUT_DIM=300


text_model=None
filename='docEmbeddings_5_clean.d2v'
if os.path.isfile(filename):
    text_model = Doc2Vec.load(filename)
else:
    text_model = Doc2Vec(min_count=1, window=5, size=Text_INPUT_DIM, sample=1e-4, negative=5, workers=4, iter=5,seed=1)
    text_model.build_vocab(sentences)
    text_model.train(sentences, total_examples=text_model.corpus_count, epochs=text_model.iter)
    text_model.save(filename)
    
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))
text_test_arrays = np.zeros((test_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = text_model.docvecs['Text_'+str(i)]

j=0
for i in range(train_size,train_size+test_size):
    text_test_arrays[j] = text_model.docvecs['Text_'+str(i)]
    j=j+1
    
print(text_train_arrays[0][:50])

[ 0.58877391  0.22714241 -0.51863343  0.95805782 -1.20313203 -0.78342247
  1.47065556 -0.79147148 -0.4045755   0.31363928  0.64597881 -0.80833185
  0.71508533 -0.39346597  0.74714977 -0.85466069  1.1426568  -1.36206746
  0.03387497 -0.68210995  1.04922903  0.22581473  0.1792437   0.7870149
  0.2268068  -0.72280908 -0.29631984  0.67567843 -0.27874869  0.08863441
  0.12197754  0.19635455 -0.46857181 -0.00280831  0.53388882 -0.21083963
 -0.12989703 -0.31110469 -0.13481045 -0.02817693 -1.849087    0.06344689
  0.03277972  0.78843325 -0.63804418  0.19981936  0.56194538 -0.20125742
 -0.45088783  0.61934203]


In [7]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_y)
encoded_y = np_utils.to_categorical((label_encoder.transform(train_y)))
print(encoded_y)

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [8]:
train_set=text_train_arrays
test_set=text_test_arrays
print(train_set[0][:50])

[ 0.58877391  0.22714241 -0.51863343  0.95805782 -1.20313203 -0.78342247
  1.47065556 -0.79147148 -0.4045755   0.31363928  0.64597881 -0.80833185
  0.71508533 -0.39346597  0.74714977 -0.85466069  1.1426568  -1.36206746
  0.03387497 -0.68210995  1.04922903  0.22581473  0.1792437   0.7870149
  0.2268068  -0.72280908 -0.29631984  0.67567843 -0.27874869  0.08863441
  0.12197754  0.19635455 -0.46857181 -0.00280831  0.53388882 -0.21083963
 -0.12989703 -0.31110469 -0.13481045 -0.02817693 -1.849087    0.06344689
  0.03277972  0.78843325 -0.63804418  0.19981936  0.56194538 -0.20125742
 -0.45088783  0.61934203]


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.optimizers import SGD

def baseline_model():
    model = Sequential()
    model.add(Dense(256, input_dim=Text_INPUT_DIM, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(180, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(9, kernel_initializer='normal', activation="softmax"))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)  
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
model = baseline_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               77056     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 180)               46260     
_________________________________________________________________
dropout_2 (Dropout)          (None, 180)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 90)                16290     
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 819       
Total params: 140,425
Trainable params: 140,425
Non-trainable params: 0
_________________________________________________________________


In [25]:
estimator=model.fit(train_set, encoded_y, validation_split=0.2, epochs=50, batch_size=64)

Train on 2656 samples, validate on 665 samples
Epoch 1/50
2656/2656 [==============================] - ETA: 0s - loss: 0.0698 - acc: 0.968 - ETA: 0s - loss: 0.0990 - acc: 0.964 - ETA: 0s - loss: 0.1116 - acc: 0.957 - ETA: 0s - loss: 0.1046 - acc: 0.961 - 0s - loss: 0.1057 - acc: 0.9601 - val_loss: 5.9027 - val_acc: 0.3203
Epoch 2/50
2656/2656 [==============================] - ETA: 0s - loss: 0.1065 - acc: 0.968 - ETA: 0s - loss: 0.0819 - acc: 0.971 - ETA: 0s - loss: 0.0926 - acc: 0.970 - ETA: 0s - loss: 0.0941 - acc: 0.967 - 0s - loss: 0.0935 - acc: 0.9676 - val_loss: 6.0550 - val_acc: 0.3038
Epoch 3/50
2656/2656 [==============================] - ETA: 0s - loss: 0.0929 - acc: 0.968 - ETA: 0s - loss: 0.1092 - acc: 0.964 - ETA: 0s - loss: 0.1025 - acc: 0.964 - 0s - loss: 0.1043 - acc: 0.9605 - val_loss: 5.6955 - val_acc: 0.3158
Epoch 4/50
2656/2656 [==============================] - ETA: 0s - loss: 0.0534 - acc: 0.984 - ETA: 0s - loss: 0.1077 - acc: 0.962 - ETA: 0s - loss: 0.1131 - acc

2656/2656 [==============================] - ETA: 0s - loss: 0.0947 - acc: 0.953 - ETA: 0s - loss: 0.0633 - acc: 0.977 - ETA: 0s - loss: 0.0708 - acc: 0.976 - ETA: 0s - loss: 0.0808 - acc: 0.971 - ETA: 0s - loss: 0.0816 - acc: 0.970 - 0s - loss: 0.0816 - acc: 0.9710 - val_loss: 5.7508 - val_acc: 0.3353
Epoch 29/50
2656/2656 [==============================] - ETA: 0s - loss: 0.0660 - acc: 0.968 - ETA: 0s - loss: 0.0827 - acc: 0.970 - ETA: 0s - loss: 0.1004 - acc: 0.964 - ETA: 0s - loss: 0.0911 - acc: 0.966 - ETA: 0s - loss: 0.0932 - acc: 0.966 - 0s - loss: 0.0940 - acc: 0.9657 - val_loss: 5.9369 - val_acc: 0.3008
Epoch 30/50
2656/2656 [==============================] - ETA: 0s - loss: 0.0713 - acc: 0.984 - ETA: 0s - loss: 0.1057 - acc: 0.962 - ETA: 0s - loss: 0.0906 - acc: 0.968 - ETA: 0s - loss: 0.0942 - acc: 0.968 - ETA: 0s - loss: 0.0955 - acc: 0.966 - 0s - loss: 0.0917 - acc: 0.9684 - val_loss: 5.7642 - val_acc: 0.3128
Epoch 31/50
2656/2656 [==============================] - ETA: 0s

In [26]:
print("Training accuracy: %.2f%% / Validation accuracy: %.2f%%" % (100*estimator.history['acc'][-1], 100*estimator.history['val_acc'][-1]))

Training accuracy: 97.14% / Validation accuracy: 32.63%
